### Import libraries

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import datetime

### Loading the New York's taxi datasets

In [0]:
df = spark.read.format('delta').load('/databricks-datasets/nyctaxi/tables/nyctaxi_yellow')
p_df = spark.read.format('parquet').load('/mnt/dls/data/big/nyctaxi_yellow_partitioned')

In [0]:
df.count()

In [0]:
p_df.count()

In [0]:
p_df.display()

In [0]:
# Define the data as lists
vendors = ['VTS', 'CMT', 'DDS', 'VTS', 'CMT', 'DDS']
payment_type = ['CASH', 'CASH', 'CASH', 'CREDIT', 'CREDIT', 'CREDIT']
extra_col = ['A', 'B', 'C', 'D', 'E', 'F']

# Define the schema of the dataframe
schema = T.StructType([
    T.StructField("vendor_id", T.StringType(), False),
    T.StructField("payment_type", T.StringType(), False),
    T.StructField("extra_col_from_m", T.StringType(), False)
])

# Create a list of tuples
data = [(vendors[i], payment_type[i], extra_col[i]) for i in range(len(vendors))]

# Create a PySpark dataframe
m_df = spark.createDataFrame(data, schema)
m_df.display()

In [0]:
p_joined_df_01 = p_df.join(other=m_df, how='inner', on = ['vendor_id', 'payment_type'])
p_joined_df_01.explain(True)

In [0]:
ts = datetime.datetime.now()
output_file_path_partitioned = '/mnt/dls/results/oscar/merged-dataset-01'
p_joined_df_01.write.format('parquet').mode('overwrite').save(output_file_path_partitioned)
p_pt = (datetime.datetime.now() - ts).seconds
print(f'The processing time was {p_pt} seconds')

### Join with hints

In [0]:
# READ: spark.sql.autoBroadcastJoinThreshold https://spark.apache.org/docs/latest/sql-performance-tuning.html#other-configuration-options
# READ: Join Hints https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-hints.html#join-hints-types
p_joined_df_02 = p_df.join(other=m_df.hint('broadcast'), how='inner', on = ['vendor_id', 'payment_type'])

In [0]:
p_joined_df_02.explain()

In [0]:
ts = datetime.datetime.now()
output_file_path_partitioned = '/mnt/dls/results/oscar/merged-dataset-02'
p_joined_df_02.write.format('parquet').mode('overwrite').save(output_file_path_partitioned)
p_pt = (datetime.datetime.now() - ts).seconds
print(f'The processing time was {p_pt} seconds')

### Join using the catalog

In [0]:
to_join_location = '/mnt/dls/results/oscar/to_join_data'
m_df.write.format('parquet').mode('overwrite').save(to_join_location)

In [0]:
spark_application_id = spark.sparkContext.applicationId.split('-')[-1]
tmp_table_name = f'{spark_application_id}_oscar_tmp_table'
tmp_table_name

In [0]:
# Register table on the spark catalog
spark.sql(f"""
CREATE TABLE {tmp_table_name} (
    vendor_id STRING,
    payment_type STRING,
    extra_col_from_m STRING
)
STORED AS PARQUET
LOCATION '{to_join_location}'
""")

In [0]:
# Make sure we obtain the metadata needed to fetch the size of this table only
spark.sql(f'ANALYZE TABLE {tmp_table_name} COMPUTE STATISTICS')

In [0]:
m_df_from_catalog = spark.sql(f'SELECT * FROM {tmp_table_name}')
m_df_from_catalog.display()

In [0]:
p_joined_df_03 = p_df.join(other=m_df_from_catalog, how='inner', on = ['vendor_id', 'payment_type'])

In [0]:
p_joined_df_03.explain()

In [0]:
ts = datetime.datetime.now()
output_file_path_partitioned = '/mnt/dls/results/oscar/merged-dataset-03'
p_joined_df_03.write.format('parquet').mode('overwrite').save(output_file_path_partitioned)
p_pt = (datetime.datetime.now() - ts).seconds
print(f'The processing time was {p_pt} seconds')

In [0]:
# Register table on the spark catalog
spark.sql(f"""DROP TABLE {tmp_table_name}""")